In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!git clone https://github.com/Arra1807/Inform_project_new.git

Cloning into 'Inform_project_new'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 167 (delta 59), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (167/167), 1.08 MiB | 2.27 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [3]:
!nvidia-smi

Wed Jul 30 12:08:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   40C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Load Data

In [ ]:
import numpy as np
from Inform_project_new.data.echosounder_data.load_data.get_echograms import get_echograms, count_classes_in_echograms

# You can select a specific year or use 'all' to include multiple years.
years = 2014
minimum_shape = 224
tuple_frequencies = (18, 38, 70, 120, 200, 333)
echograms = get_echograms(years=years, tuple_frequencies=tuple_frequencies, minimum_shape=minimum_shape)
print(f"Number of echograms: {len(echograms)}")



# Split Large Echograms into Train images and Test images

In [ ]:
Train_eg = []
test_eg = []
for i,echogram in enumerate(echograms):
  if i < 8:
    Train_eg.append(echogram)
  else:
    test_eg.append(echogram)
print('Train Echogram:', len(Train_eg))
print('Test Echogram:', len(test_eg))

In [ ]:
from Inform_project_new.data.echosounder_data.preprocessing.resize_and_crop import SplitResizeEchogram, generate_patch_batches
from Inform_project_new.data.echosounder_data.preprocessing.normalization import db, clip, standardize_min_max
from Inform_project_new.adaptation_method.model_config import Configuration

#For logging purposes
run= Configuration()

data_transform = [db, clip, standardize_min_max]
split_patch_size = 224
output_patch_size = 224


#---Train set---
train_generators, num_patches_per_train_echogram = generate_patch_batches(Train_eg, split_patch_size,
                                                                          output_patch_size, data_transforms= data_transform,
                                                                          batch_size='full', verbose= False)
print('Number of patches per echogram for training:', num_patches_per_train_echogram )

#---Test set---
generators_test, num_patches_per_test_echogram = generate_patch_batches(test_eg, split_patch_size, output_patch_size,
                                                                        data_transforms= data_transform,
                                                                        batch_size='full', verbose= False)
print('Number of patches per echogram for testing:', num_patches_per_train_echogram )

# Dataloader

In [ ]:
from Inform_project_new.data.echosounder_data.dataloader import BatchGeneratorDataset, GroupedGenerator, group_generators_by_patch_limit
from torch.utils.data import DataLoader
from Inform_project_new.adaptation_method.model_config import Configuration

#---Training set---
print('-------Dimensions for Training set-------------')
grouped_generator_fns, grouped_patch_count = group_generators_by_patch_limit(train_generators, num_patches_per_train_echogram, 64)

train_dataset = BatchGeneratorDataset(grouped_generator_fns)
#train_loader = DataLoader(train_dataset, batch_size = None, shuffle= False)

for labels, data, mask in train_dataset:
  print("Data shape:", data.shape)

#---Test Set---
print('-------Dimensions for Test set-------------')
grouped_generator_fns, grouped_patch_count = group_generators_by_patch_limit(generators_test, num_patches_per_test_echogram, 64)

test_dataset = BatchGeneratorDataset(grouped_generator_fns)
#test_loader = DataLoader(test_dataset, batch_size = None, shuffle= False)

for labels, data, mask in test_dataset:
  print("Data shape:", data.shape)




In [ ]:
#from adaptation_method.train_autoencoder import train_val_encoder
from Inform_project_new.adaptation_method.train_autoencoder import train_val_encoder, plot_loss
from Inform_project_new.adaptation_method.Autoencoder import ResNetAutoEncoder
import torch.optim as optim
import torch.nn as nn

#Instantiating model
model = ResNetAutoEncoder()

# Defining Loss function and optimizer
Loss_func = nn.MSELoss()

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                       lr= run.config['learning_rate'],
                       weight_decay= run.config['Weight_decay']
)

encoded, avg_loss_train, val_encoded, avg_loss_val, epoch_stop = train_val_encoder(model, optimizer, Loss_func, run.config['epochs'], train_dataset, test_dataset, run)

plot_loss(run.config['epochs'], avg_loss_train, avg_loss_val, epoch_stop)

In [ ]:
from Inform_project_new.adaptation_method.train_autoencoder import train_val_encoder, plot_loss
from Inform_project_new.adaptation_method.Autoencoder.py import Autoencoder
import torch.nn as nn
import torch.optim as optim

#Instantiating model
model = Autoencoder()

# Defining Loss function and optimizer
Loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = run.config['learning_rate'], weight_decay = run.config['Weight_decay'])

encoded, avg_loss_train, val_encoded, avg_loss_val, stop_epoch = train_val_encoder(model, optimizer, Loss_func, run.config['epochs'], train_loader, test_loader, run)

plot_loss(run.config['epochs'], avg_loss_train, avg_loss_val, stop_epoch)


In [ ]:
encoder = encoded.squeeze(0)
encoder_transpose = encoder.permute(2,1,0)
encoder = encoder_transpose.detach().numpy()
print(encoder.shape)